<a href="https://colab.research.google.com/github/toka-amer/Sport-Image-Classification/blob/main/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
import os
from tqdm import tqdm
from google.colab import drive
from google.colab.patches import cv2_imshow
import cv2
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
!pip install tflearn
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import matplotlib as plt
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import keras
import keras.backend as K
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils
import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from random import shuffle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 8.1 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=75d9286311c5bd2875f315658c4ed46676de10bdab0e0201aa84a4a1d3aab0f0
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
drive.mount('/content/MyDrive/',force_remount = True)

Mounted at /content/MyDrive/


In [ ]:
 !unzip '/content/MyDrive/MyDrive/NN Dataset.zip'

In [ ]:
def imshow(img,transform):
  img = PIL.Image.open(img)
  print(img)
  fig,ax = plt.subplots(1,2,figsize = (15,4))
  ax[0].set_title(f'original image {img.size}')
  ax[0].imshow(img)
  img = transform(img)
  ax[1].set_title(f'tansformed image {img.size}')
  ax[1].imshow(img)

In [ ]:
TRAIN_DIR = "/content/Train"
TEST_DIR = '/content/Test'
IMG_SIZE = 224
LR = 0.001
MODEL_NAME = 'AlexNet'

Basketball = []
Football = [] 
Rowing = []
Swimming = []
Tennis = []
Yoga = []

for img in tqdm(os.listdir(TRAIN_DIR)):
  path = os.path.join(TRAIN_DIR,img)
  image = cv2.imread(path,1)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
  if img.__contains__("Basketball"):
    Basketball.append([np.array(image), np.array([1,0,0,0,0,0])])
  elif img.__contains__("Football"):
    Football.append([np.array(image), np.array([0,1,0,0,0,0])])
  elif img.__contains__("Rowing"):
    Rowing.append([np.array(image), np.array([0,0,1,0,0,0])])
  elif img.__contains__("Swimming"):
    Swimming.append([np.array(image), np.array([0,0,1,0,0,0])])
  elif img.__contains__("Tennis"):
    Tennis.append([np.array(image), np.array([0,0,0,0,1,0])])
  elif img.__contains__("Yoga"):
    Yoga.append([np.array(image), np.array([0,0,0,0,0,1])])
print(" ")
print("number of each class")
print(len(Basketball))
print(len(Football))
print(len(Rowing))
print(len(Swimming))
print(len(Tennis))
print(len(Yoga))

####balance dataset by flip horizantal######
def augmentationFlipHorizontal(data,name):
  for img in data:
    if len(data) != 458:
      flappedImage = cv2.flip(image,1)
      if name == "Basketball":
        Basketball.append([np.array(flappedImage), np.array([1,0,0,0,0,0])])
      elif name == "Football":
        Football.append([np.array(flappedImage), np.array([0,1,0,0,0,0])])
      elif name == "Rowing":
        Rowing.append([np.array(flappedImage), np.array([0,0,1,0,0,0])])
      elif name == "Swimming":
        Swimming.append([np.array(flappedImage), np.array([0,0,0,1,0,0])])
      elif name == "Tennis":
        Tennis.append([np.array(flappedImage), np.array([0,0,0,0,1,0])])
      elif name == "Yoga":
        Yoga.append([np.array(flappedImage), np.array([0,0,0,0,0,1])])

augmentationFlipHorizontal(Basketball,"Basketball")
augmentationFlipHorizontal(Football,"Football")
augmentationFlipHorizontal(Rowing,"Rowing")
augmentationFlipHorizontal(Swimming,"Swimming")
augmentationFlipHorizontal(Tennis,"Tennis")
augmentationFlipHorizontal(Yoga,"Yoga")
print("number of each class after augmentation")
print(len(Basketball))
print(len(Football))
print(len(Rowing))
print(len(Swimming))
print(len(Tennis))
print(len(Yoga))
def create_label(image_name):
    """ Create an one-hot encoded vector from image name """
    word_label = image_name.split('_')[0]
    if word_label == 'Basketball':
        return np.array([1,0,0,0,0,0])
    elif word_label == 'Football':
        return np.array([0,1,0,0,0,0])
    elif word_label == 'Rowing':
        return np.array([0,0,1,0,0,0])
    elif word_label == 'Swimming':
        return np.array([0,0,0,1,0,0])
    elif word_label == 'Tennis':
        return np.array([0,0,0,0,1,0])
    elif word_label == 'Yoga':
        return np.array([0,0,0,0,0,1])


train = []
def createTrainData(data):
  for i in data:
    train.append(i)


createTrainData(Basketball)
createTrainData(Football)
createTrainData(Rowing)
createTrainData(Swimming)
createTrainData(Tennis)
createTrainData(Yoga)

shuffle(train)
print("shape of train data")
X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y_train = np.array([i[1] for i in train])
print(X_train.shape)
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

100%|██████████| 1681/1681 [00:07<00:00, 236.90it/s]


 
number of each class
196
400
202
240
185
458
number of each class after augmentation
458
458
458
458
458
458
shape of train data
(2748, 224, 224, 3)


In [ ]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output



In [ ]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [ ]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(6, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(6, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(6, activation='softmax', name='output')(x)

In [ ]:

model = Model(input_layer, [x, x1, x2], name='inception_v1')


In [ ]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, [y_train, y_train, y_train], validation_data=(X_test, [y_test, y_test, y_test]), epochs=25, batch_size=256, callbacks=[lr_sc])
model.save('inception.tfl')

Train on 2061 samples, validate on 687 samples

Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/25
2061/2061 [==============================] - 7s 3ms/sample - loss: 0.7250 - output_loss: 0.4492 - auxilliary_output_1_loss: 0.4589 - auxilliary_output_2_loss: 0.4555 - output_acc: 0.8333 - auxilliary_output_1_acc: 0.8328 - auxilliary_output_2_acc: 0.8321 - val_loss: 0.7078 - val_output_loss: 0.4428 - val_auxilliary_output_1_loss: 0.4428 - val_auxilliary_output_2_loss: 0.4412 - val_output_acc: 0.8333 - val_auxilliary_output_1_acc: 0.8333 - val_auxilliary_output_2_acc: 0.8333 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/25
2061/2061 [==============================] - 7s 3ms/sample - loss: 0.7192 - output_loss: 0.4478 - auxilliary_output_1_loss: 0.4539 - auxilliary_output_2_loss: 0.4547 - output_acc: 0.8333 - auxilliary_output_1_acc: 0.8329 - auxilliary_output_2_acc: 0.8332 - val_loss: 0.7040 - val_output_loss: 0.4399 - val_auxilli